# Decoding Strategies

## Initialization

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline, set_seed
import torch

In [4]:
set_seed(42)

## Models

In [5]:
tokenizer = AutoTokenizer.from_pretrained("SZTAKI-HLT/mT5-base-HunSum-1")
model = AutoModelForSeq2SeqLM.from_pretrained("SZTAKI-HLT/mT5-base-HunSum-1")

In [7]:
tokenizer = AutoTokenizer.from_pretrained("SZTAKI-HLT/mT5-base-HunSum-1")
model = AutoModelForSeq2SeqLM.from_pretrained("SZTAKI-HLT/mT5-base-HunSum-1")

## Currently Used (Beam Search)

In [9]:
nlp = pipeline(model=model, task='summarization', tokenizer=tokenizer)
nlp(prompt,
    max_length=128,
    num_beams=5,
    length_penalty=2,
    no_repeat_ngram_size=2,
    encoder_no_repeat_ngram_size=3,
    early_stopping=True
)

[{'summary_text': 'A magyar búvárválogatott bronzérmet nyert a csehországi Jablonecban zajló tájékozódási úszó-Eb-n, ezzel második lett.'}]

## Greedy Search

Required parameters:
- num_beams = 0
- do_sample = False

In [8]:
output = model.generate(
    input_ids,
    num_beams=1,
    do_sample=False,
    max_length=128,
    no_repeat_ngram_size=2
)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("generated_text: ", generated_text)

generated_text:  A magyar búvárválogatott bronzérmet nyert a szolnoki Európa-bajnokságon.


## Multinomial sampling

Required parameters:
- do_sample = True

In [7]:
output = model.generate(
    input_ids,
    num_beams=1,
    do_sample=True,
    max_length=128,
    no_repeat_ngram_size=2
)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("generated_text: ", generated_text)

generated_text:  Bronzérmes lett a búvárúszók szolnoki Európa-bajnokságán döntős magyar csapat, melynek összesítésében tizenketten győztek.


## Beam-search multinomial sampling

Required parameters:
- do_sample = True
- num_beams > 1

In [6]:
output = model.generate(
    input_ids,
    num_beams=5,
    do_sample=True,
    max_length=128,
    no_repeat_ngram_size=2
)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("generated_text: ", generated_text)

generated_text:  A búvárúszók Európa-bajnokságán a magyar válogatott bronzérmet szerzett.


## Diverse beam search

Required parameters:
- num_beams > 1
- num_beam_groups > 1

In [5]:
output = model.generate(
    input_ids,
    num_beams=5,
    num_beam_groups=5,
    do_sample=False,
    max_new_tokens=128,
    no_repeat_ngram_size=2,
    diversity_penalty=1.0
)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("generated_text: ", generated_text)

C:\Users\nyist\miniconda3\envs\summ\lib\site-packages\transformers\generation\beam_search.py:198: UserWarning: Passing `max_length` to BeamSearchScorer is deprecated and has no effect. `max_length` should be passed directly to `beam_search(...)`, `beam_sample(...)`, or `group_beam_search(...)`.
  warnings.warn(


generated_text:  A magyar búvárválogatott bronzérmet nyert a szolnoki Európa-bajnokságon.


## Contrastive search

Required parameters:
- penalty_alpha
- top_k

In [4]:
output = model.generate(
    input_ids,
    num_beams=1,
    do_sample=False,
    penalty_alpha=0.6,
    top_k=4,
    max_length=128
)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("generated_text: ", generated_text)

generated_text:  A búvárúszók németországi Európa-bajnokságán a felnőttek mellett a juniorok is bronzérmesek lettek.


## Top-k Sampling

In [7]:
output = model.generate(
    input_ids,
    num_beams=1,
    max_length=128,
    do_sample=True,
    top_k=50,
    no_repeat_ngram_size=3,
    encoder_no_repeat_ngram_size=4
)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("generated_text: ", generated_text)

generated_text:  Bronzérmes lett a magyar férfi vízilabda-válogatott a szlovéniai Jablonecban zajló Európa-bajnoki selejtezőverseny egyetlen szerdai napján, amelyen már a juniorokat is a dobogó legfelszavazása segítette.


## Top-P Sampling

In [8]:
output = model.generate(
    input_ids,
    num_beams=1,
    max_length=128,
    do_sample=True,
    top_p=0.92,
    top_k=0,
    no_repeat_ngram_size=3,
    encoder_no_repeat_ngram_size=4
)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("generated_text: ", generated_text)

generated_text:  A lányok közül a Csépő Kelen, Horváth Ferenc, Borbély János összeállítású páros is a legjobban szerepelt a magyar búvárúszóválogatott szombat délelőtti bronzérmes versenyén.


## Temperature Sampling

In [9]:
output = model.generate(
    input_ids,
    do_sample=True,           
    temperature=0.7,
    max_length=128,
    no_repeat_ngram_size=3,
    encoder_no_repeat_ngram_size=4
)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("generated_text: ", generated_text)

generated_text:  A magyar búvárcsapat bronzérmet szerzett a szolnoki Európa-bajnoki búvárúszás döntőjében, amelyen a férfiak mellett ugyancsak aranyérmet nyertek a junior kategóriában.


### Sources:
- https://huggingface.co/docs/transformers/v4.27.2/en/generation_strategies
- https://huggingface.co/blog/how-to-generate
- https://arxiv.org/abs/2202.06417
- https://arxiv.org/pdf/1610.02424.pdf